# DecisionTreeClassifer


#Imports and Dataset processing

In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import metrics
import copy
from sympy import Symbol
from sympy.solvers import solve
pd.options.mode.chained_assignment = None
import csv

In [26]:
url = "https://raw.githubusercontent.com/niceIrene/remedy/main/datasets/bar_pass_prediction.csv"
data = pd.read_csv(url)
# data = data.drop(columns = ['poutcome'])
# data = data.drop(columns = ['ID', ''])
# data.dtypes
data = data.drop(columns = ['ID', 'decile1b', 'pass_bar', 'other', 'gpa', 'zgpa', 'zfygpa',  'cluster',  'bar1_yr', 'bar2', 'bar2_yr', 'bar_passed','race', 'race2', 'asian', 'black', 'hisp', 'Dropout', 'male', 'bar', 'parttime', 'index6040', 'indxgrp', 'indxgrp2', 'dnn_bar_pass_prediction', 'DOB_yr'])

# data['bar_passed'].value_counts()
data

,decile3,decile1,sex,lsat,ugpa,grad,bar1,fulltime,fam_inc,age,gender,race1,tier
0,10.0,10.0,1.0,44.0,3.5,Y,P,1.0,5.0,-62.0,female,white,4.0
1,4.0,5.0,1.0,29.0,3.5,Y,P,1.0,4.0,-62.0,female,white,2.0
2,2.0,3.0,2.0,36.0,3.5,Y,P,1.0,1.0,-58.0,male,white,3.0
3,4.0,7.0,2.0,39.0,3.5,Y,P,1.0,4.0,-51.0,male,white,3.0
4,8.0,9.0,2.0,48.0,3.5,Y,P,1.0,4.0,-61.0,male,white,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22402,1.0,3.0,2.0,26.5,1.8,Y,F,1.0,2.0,-62.0,male,black,1.0
22403,1.0,3.0,2.0,19.7,1.8,Y,F,1.0,3.0,-57.0,male,black,1.0
22404,8.0,7.0,2.0,36.0,1.8,Y,P,2.0,3.0,-59.0,male,black,3.0
22405,10.0,10.0,2.0,44.0,1.5,Y,P,2.0,3.0,-51.0,male,white,3.0


In [27]:
data = data.dropna()

In [28]:
data["bar1"].value_counts()
# data["ugpa"].describe()

P    18140
F     2287
Name: bar1, dtype: int64

In [29]:
# # make it a balanced dataset


data["bar1"].value_counts()

rows_to_remove = data[data['bar1'] == 'P'].sample(n=15853).index
data = data.drop(rows_to_remove)


data

,decile3,decile1,sex,lsat,ugpa,grad,bar1,fulltime,fam_inc,age,gender,race1,tier
9,5.0,7.0,1.0,28.0,3.5,Y,F,1.0,2.0,-62.0,female,hisp,1.0
10,2.0,3.0,1.0,33.0,3.5,Y,F,1.0,4.0,-65.0,female,white,2.0
11,5.0,6.0,1.0,36.0,3.5,Y,P,1.0,3.0,-62.0,female,white,4.0
17,3.0,5.0,1.0,33.0,3.5,Y,F,1.0,4.0,-62.0,female,white,5.0
28,7.0,7.0,1.0,30.0,3.5,Y,F,1.0,3.0,-61.0,female,white,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22393,1.0,2.0,1.0,39.5,1.7,Y,F,1.0,4.0,-54.0,female,white,3.0
22400,6.0,9.0,2.0,31.0,1.8,Y,F,1.0,2.0,-53.0,male,black,1.0
22401,1.0,1.0,2.0,25.0,1.8,Y,F,1.0,3.0,-46.0,male,other,2.0
22402,1.0,3.0,2.0,26.5,1.8,Y,F,1.0,2.0,-62.0,male,black,1.0


In [24]:
# data["lsat"] = pd.cut(data["lsat"], [0,33,37,41,50], labels=["<33", "33-37","37-41",">41"])
# data["ugpa"] = pd.cut(data["ugpa"], [0,3,3.2,3.5,4], labels=["<3", "3-3.2","3.2-3.5",">3.5"])
# data



,decile3,decile1,sex,lsat,ugpa,grad,bar1,fulltime,fam_inc,age,gender,race1,tier
9,5.0,7.0,1.0,<33,3.2-3.5,Y,F,1.0,2.0,-62.0,female,hisp,1.0
10,2.0,3.0,1.0,<33,3.2-3.5,Y,F,1.0,4.0,-65.0,female,white,2.0
11,5.0,6.0,1.0,33-37,3.2-3.5,Y,P,1.0,3.0,-62.0,female,white,4.0
17,3.0,5.0,1.0,<33,3.2-3.5,Y,F,1.0,4.0,-62.0,female,white,5.0
24,9.0,10.0,2.0,33-37,3.2-3.5,Y,P,1.0,4.0,-61.0,male,white,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22393,1.0,2.0,1.0,37-41,<3,Y,F,1.0,4.0,-54.0,female,white,3.0
22400,6.0,9.0,2.0,<33,<3,Y,F,1.0,2.0,-53.0,male,black,1.0
22401,1.0,1.0,2.0,<33,<3,Y,F,1.0,3.0,-46.0,male,other,2.0
22402,1.0,3.0,2.0,<33,<3,Y,F,1.0,2.0,-62.0,male,black,1.0


In [30]:
data = data.drop(columns=["age"], axis = 1)
data

,decile3,decile1,sex,lsat,ugpa,grad,bar1,fulltime,fam_inc,gender,race1,tier
9,5.0,7.0,1.0,28.0,3.5,Y,F,1.0,2.0,female,hisp,1.0
10,2.0,3.0,1.0,33.0,3.5,Y,F,1.0,4.0,female,white,2.0
11,5.0,6.0,1.0,36.0,3.5,Y,P,1.0,3.0,female,white,4.0
17,3.0,5.0,1.0,33.0,3.5,Y,F,1.0,4.0,female,white,5.0
28,7.0,7.0,1.0,30.0,3.5,Y,F,1.0,3.0,female,white,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
22393,1.0,2.0,1.0,39.5,1.7,Y,F,1.0,4.0,female,white,3.0
22400,6.0,9.0,2.0,31.0,1.8,Y,F,1.0,2.0,male,black,1.0
22401,1.0,1.0,2.0,25.0,1.8,Y,F,1.0,3.0,male,other,2.0
22402,1.0,3.0,2.0,26.5,1.8,Y,F,1.0,2.0,male,black,1.0


In [31]:
data.to_csv('bar_pass.csv', index = False)

In [32]:
data.dtypes

decile3     float64
decile1     float64
sex         float64
lsat        float64
ugpa        float64
grad         object
bar1         object
fulltime    float64
fam_inc     float64
gender       object
race1        object
tier        float64
dtype: object

In [9]:
# data['age'] = data['age'].astype(str)
# for c in data.columns:
#     print(c)
#     if type(data[c].unique()[0]) == str:
#       print(data[c].unique())
#       temp = list(data[c].unique())
#       print(temp)
#       for t in range(len(temp)):
#           print(str(t), temp[t])
#           data[c] = data[c].str.replace(temp[t],str(t))
#     # if c == 'age':

#     #   temp = list(data[c].unique())
#     #   for a in range(len(temp)):
#     #     data[c].str.replace(temp[a], str(a))

# data

In [ ]:
# data['age'] = data['age'].astype(int)

# age_bins = [0, 10, 20, 30, 40, 50]  # Define your own bin edges
# age_labels = [0, 1, 2, 3, 4]
# data['age'] = pd.cut(data['age'], bins=age_bins, labels=age_labels, include_lowest=True)
# data['age'].value_counts()

In [ ]:
# data["gender"] = pd.to_numeric(data["gender"])
# data["age"] = pd.to_numeric(data["age"])
# data["race1"] = pd.to_numeric(data["race1"])

In [ ]:
# data.dtypes

In [ ]:
# data

In [ ]:
# data.columns

In [7]:
data["bar1"].value_counts()

F    2287
P    2287
Name: bar1, dtype: int64

In [ ]:
# data["bar1"] = data["bar1"].astype(int)
# data.dtypes

In [ ]:
# data.to_csv('bar_pass.csv', index = False)

# DATA NEW

In [ ]:
# Yin added
url = "https://raw.githubusercontent.com/niceIrene/remedy/main/datasets/bar_pass_new.csv"
data = pd.read_csv(url)
data = data.drop(columns=["gender"])
data

,decile1b,decile3,decile1,sex,lsat,ugpa,grad,bar1,fulltime,fam_inc,age,race1,tier
0,3.0,2.0,3.0,2.0,0,0,0,0,1.0,1.0,0,0,3.0
1,7.0,5.0,7.0,1.0,1,0,0,1,1.0,2.0,0,1,1.0
2,2.0,2.0,3.0,1.0,1,0,0,1,1.0,4.0,0,0,2.0
3,5.0,3.0,5.0,1.0,1,0,0,1,1.0,4.0,0,0,5.0
4,5.0,5.0,5.0,1.0,2,0,0,0,1.0,3.0,0,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4500,9.0,6.0,9.0,2.0,1,1,0,1,1.0,2.0,2,2,1.0
4501,1.0,1.0,1.0,2.0,1,1,0,1,1.0,3.0,2,4,2.0
4502,3.0,1.0,3.0,2.0,1,1,0,1,1.0,2.0,0,2,1.0
4503,2.0,1.0,3.0,2.0,1,1,0,1,1.0,3.0,1,2,1.0


In [ ]:
# get training and testing set

# columns_compas = ['stay', 'age', 'charge', 'sex', '#prior', 'race']


compas_y = 'bar1'
columns_all =['decile1b', 'decile3', 'decile1', 'sex', 'lsat', 'ugpa', 'grad',
       'fulltime', 'fam_inc', 'race1', 'tier', 'age']
# columns_compas =['fam_inc', 'race1', 'lsat', 'ugpa', 'sex', 'fulltime']
columns_compas = ["sex", "age", "race1", "fam_inc"]
# columns_compas = ['decile1b', 'decile3', 'decile1', 'sex', 'lsat', 'ugpa', 'grad',
#        'fulltime', 'fam_inc', 'gender', 'race1', 'tier']
def split_train_test(data,test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices],data.iloc[test_indices]

def get_train_test(data, split, list_cols, y_label):
  train_set,test_set = split_train_test(data,split)
  print(len(train_set), "train +", len(test_set), "test")
  train_x = pd.DataFrame(train_set, columns = list_cols)
  train_label = train_set[y_label]
  test_x = pd.DataFrame(test_set, columns = list_cols)
  test_label = test_set[y_label]
  return train_x, test_x, train_label, test_label, train_set, test_set

In [ ]:
columns_compas


['sex', 'age', 'race1', 'fam_inc']

In [ ]:
train_x, test_x, train_label, test_label, train_set, test_set  = get_train_test(data, 0.3, columns_all, compas_y)

###################

### about decision tree
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

param = {'criterion': ['gini', 'entropy'], 'max_depth': [10, 20, 30, 40, 50, 100], 'random_state':[17]}

grid = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
grid.fit(train_x, train_label)

print(list(train_label))
# print("best", grid.best_score_)
# test_predict = grid.predict(test_x)
data_all = pd.concat([train_x,test_x])
data_predict = grid.predict(data_all)
test_predict = grid.predict(test_x)
data['predicted'] = data_predict

test_set['predicted'] = test_predict
print(test_set)
print(data[compas_y].value_counts())
print(data['predicted'].value_counts())
print(test_label.value_counts())
print(test_predict)

3154 train + 1351 test
[1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0

In [ ]:
pip install divexplorer

In [ ]:
def fpr_onegroup(true, predict):
    fp = 0
    tn = 0
    for i in range(len(true)):
        if (true[i] == 0 and predict[i] == 1):
            fp += 1
        if(true[i] == 0 and predict[i] == 0):
            tn += 1
    print(fp, tn)
    return fp/(fp+tn)


def fnr_onegroup(true, predict):
    fn = 0
    tp = 0
    for i in range(len(true)):
        if (true[i] == 1 and predict[i] == 0):
            fn += 1
        if(true[i] == 1 and predict[i] == 1):
            tp += 1
    print(fn, tp)
    return fn/(fn+tp)

In [ ]:
fpr = fpr_onegroup(list(test_label), test_predict)
print("fpr is " , fpr)

fnr = fnr_onegroup(list(test_label), test_predict)
print("fnr is " , fnr)

148 523
fpr is  0.22056631892697467
193 487
fnr is  0.2838235294117647


In [ ]:
# print(fpr_onegroup(l1, l3))
accuracy = accuracy_score(test_label, test_predict)
print("accuracy is " , accuracy)


accuracy is  0.7475943745373798


In [ ]:
def fairness_score_computation(d, metrics):
    sum_of_score = 0
    for idx, row in d.iterrows():
      sum_of_score += row['support'] * row[metrics]
    return sum_of_score


# print(fairness_score_computation(d, 'd_fpr'))
# print(fairness_score_computation(d2, 'd_fnr'))


In [ ]:
filter_count = 30

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, make_scorer
filter_count = 1

# ORIGINAL

scoring = make_scorer(accuracy_score)
param_griddt = {
    'max_depth': [2, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dt = DecisionTreeClassifier(random_state=42)
griddt = GridSearchCV(dt, param_grid=param_griddt, scoring=scoring, cv=5)
svc = SVC(probability=True, kernel="poly", degree = 2)
gridrf = GridSearchCV(RandomForestClassifier(), param_grid = param, cv = 6)
paraml = {'penalty': ['l2'],'C':[0.1, 1, 10, 100, 1000], 'solver':['saga'], 'multi_class':['ovr']}
gridl = GridSearchCV(LogisticRegression(), param_grid = paraml, cv = 6)

In [ ]:
# type(test_x["age"].iloc[0])

In [ ]:
# ORIGINAL HERE
gridl.fit(train_x, train_label)
test_predict = gridl.predict(test_x)
test_set['predicted'] = test_predict
accuracy_score(test_label, test_set['predicted'])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

0.7809030347890451

In [ ]:
class_map={'N': 0, 'P': 1}
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer
columns_compas.extend([compas_y, "predicted"])

df = pd.DataFrame(test_set, columns = columns_compas)
# print(test_set.columns)
# print(columns_compas)
# print(df.columns)
# print(columns_compas)

columns_compas.remove(compas_y)
columns_compas.remove('predicted')

######

min_sup=0.1
# min_sup = 0.05
# min_sup = 0.01

fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
from divexplorer.FP_Divergence import FP_Divergence
fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]
eps=0.01
K=1000
d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3]



pd.options.display.max_rows = 200
d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# summerization

d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d= d[d['d_fpr'] > 0]
d2= d2[d2['d_fnr'] > 0]
d3= d3[d3['d_accuracy'] > 0]

dfpr = fairness_score_computation(d, 'd_fpr')
dfnr = fairness_score_computation(d2, 'd_fnr')
dacc = fairness_score_computation(d3, 'd_accuracy')
"lg", dfpr, dfnr, dacc

ModuleNotFoundError: ignored

In [ ]:
d

In [ ]:
griddt.fit(train_x, train_label)
test_predict = griddt.predict(test_x)
test_set['predicted'] = test_predict
accuracy_score(test_label, test_set['predicted'])

In [ ]:
class_map={'N': 0, 'P': 1}
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer
columns_compas.extend([compas_y, "predicted"])

df = pd.DataFrame(test_set, columns = columns_compas)
# print(test_set.columns)
# print(columns_compas)
# print(df.columns)
# print(columns_compas)

columns_compas.remove(compas_y)
columns_compas.remove('predicted')

######

min_sup=0.1
# min_sup = 0.05
# min_sup = 0.01

fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
from divexplorer.FP_Divergence import FP_Divergence
fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]
eps=0.01
K=1000
d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3]



pd.options.display.max_rows = 200
d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# summerization

d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d= d[d['d_fpr'] > 0]
d2= d2[d2['d_fnr'] > 0]
d3= d3[d3['d_accuracy'] > 0]

dfpr = fairness_score_computation(d, 'd_fpr')
dfnr = fairness_score_computation(d2, 'd_fnr')
dacc = fairness_score_computation(d3, 'd_accuracy')
"dt", dfpr, dfnr, dacc

In [ ]:
d

In [ ]:
gridrf.fit(train_x, train_label)
test_predict = gridrf.predict(test_x)
test_set['predicted'] = test_predict
accuracy_score(test_label, test_set['predicted'])

In [ ]:
class_map={'N': 0, 'P': 1}
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer
columns_compas.extend([compas_y, "predicted"])

df = pd.DataFrame(test_set, columns = columns_compas)
# print(test_set.columns)
# print(columns_compas)
# print(df.columns)
# print(columns_compas)

columns_compas.remove(compas_y)
columns_compas.remove('predicted')

######

min_sup=0.1
# min_sup = 0.05
# min_sup = 0.01

fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
from divexplorer.FP_Divergence import FP_Divergence
fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]
eps=0.01
K=1000
d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3]



pd.options.display.max_rows = 200
d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# summerization

d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d= d[d['d_fpr'] > 0]
d2= d2[d2['d_fnr'] > 0]
d3= d3[d3['d_accuracy'] > 0]

dfpr = fairness_score_computation(d, 'd_fpr')
dfnr = fairness_score_computation(d2, 'd_fnr')
dacc = fairness_score_computation(d3, 'd_accuracy')
"rf", dfpr, dfnr, dacc

In [ ]:
d

In [ ]:
svc.fit(train_x, train_label)
test_predict = svc.predict(test_x)
test_set['predicted'] = test_predict
accuracy_score(test_label, test_set['predicted'])

In [ ]:
class_map={'N': 0, 'P': 1}
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer
columns_compas.extend([compas_y, "predicted"])

df = pd.DataFrame(test_set, columns = columns_compas)
# print(test_set.columns)
# print(columns_compas)
# print(df.columns)
# print(columns_compas)

columns_compas.remove(compas_y)
columns_compas.remove('predicted')

######

min_sup=0.1
# min_sup = 0.05
# min_sup = 0.01

fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
from divexplorer.FP_Divergence import FP_Divergence
fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]
eps=0.01
K=1000
d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3]



pd.options.display.max_rows = 200
d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# summerization

d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d= d[d['d_fpr'] > 0]
d2= d2[d2['d_fnr'] > 0]
d3= d3[d3['d_accuracy'] > 0]

dfpr = fairness_score_computation(d, 'd_fpr')
dfnr = fairness_score_computation(d2, 'd_fnr')
dacc = fairness_score_computation(d3, 'd_accuracy')
"svc", dfpr, dfnr, dacc

In [ ]:
d

# Divexplorer

In [ ]:
# run divexplorer to find unfair groups
class_map={'N': 0, 'P': 1}
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer


######
# Yin: by consideri the unfair subgroups, we only consider the subgroups given by the protected attributes.

columns_compas.extend([compas_y, "predicted"])

df = pd.DataFrame(test_set, columns = columns_compas)
# print(test_set.columns)
print(columns_compas)
print(df.columns)
print(columns_compas)

columns_compas.remove(compas_y)
columns_compas.remove('predicted')

######

min_sup=0.1
# min_sup = 0.05
# min_sup = 0.01

fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
from divexplorer.FP_Divergence import FP_Divergence
fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]
eps=0.01
K=1000
d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ].head(K)
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2].head(K)
d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3].head(K)

# dfpr = fairness_score_computation(d, 'd_fpr')
# dfnr = fairness_score_computation(d2, 'd_fnr')
# dacc = fairness_score_computation(d3, 'd_accuracy')

pd.options.display.max_rows = 200
d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# summerization

d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ].head(K)
d= d[d['d_fpr'] > 0]
d, list(d["itemsets"].iloc[0])


In [ ]:

d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2].head(K)
d2= d2[d2['d_fnr'] > 0]
d2, list(d2["itemsets"].iloc[0])

In [ ]:
# with open('GC_results.csv', 'w', newline='') as file:
#   writer = csv.writer(file)
#   writer.writerow(["Dataset","Remedy", "Algorithm","d_fpr","d_fnr", "d_acc", "model_acc"])
#   writer.writerow(["German Credit", "Original", "None", dfpr, dfnr, dacc, "None"])

# For entire dataset


In [ ]:
import itertools
def get_unfair_group(list_parse, entire = 1):
  unfair_group = []
  unfair_dict = {}
  names = []
  for col in columns_compas:
    found = False
    for item in list_parse:
      attr_given = item.split("=")[0]
      if col == attr_given:
        unfair_group.append(int(item.split("=")[1]))
        names.append(attr_given)
        unfair_dict[attr_given] = int(item.split("=")[1])
        found = True
  # if use the entire dataset
  if entire:
    return unfair_group, names, columns_compas, unfair_dict
        # break
    # if found == False:
    #   unfair_group.append(-1)
  return unfair_group, names, list(set(columns_compas).symmetric_difference(set(names))), unfair_dict
def candidate_groups(skew_candidates, unfair_dict, ordering, names):
  candidate_combos = []
  candidate_ind = {}
  num = 0
  for i in range(len(skew_candidates)+1):
    temp_candidate = list(itertools.combinations(skew_candidates, i))
    for tc in temp_candidate:
      candidate_ind[num] = list(tc)
      num += 1
  return candidate_ind

def name_val_dict(train_set,names):
  names_values = {}
  for n in names:
    names_values[n] = list(train_set[n].unique())
  return names_values

In [ ]:
def get_temp(train_set, names, y_label):
  names2 = copy.deepcopy(names)
  names2.append(y_label)
  temp = train_set[names2]
  temp['cnt'] = 0
  temp2 = temp.groupby(names2)['cnt'].count().reset_index()
  temp2['cnt'].sum()
  return temp2, names
temp2, names = get_temp(train_set, columns_compas, compas_y)
temp2

In [ ]:
def get_temp_g(train_set, names, y_label):
  names2 = copy.deepcopy(names)
  names2.append(y_label)
  temp = train_set[names2]
  temp['cnt'] = 0
  temp_g = temp.groupby(names)['cnt'].count().reset_index()
  return temp, temp_g

In [ ]:
unfair_group, unfair_names, skew_candidates, unfair_dict = get_unfair_group([])
print(unfair_group, unfair_names, skew_candidates, unfair_dict)
all_names = candidate_groups(skew_candidates, unfair_dict, columns_compas, unfair_names)
names_values = name_val_dict(train_set, names)
print(all_names[len(all_names)-1])
print(names_values)
all_names_lst = list(all_names.keys())[1:] # CHANGED HERE
all_names_lst.reverse()
print(all_names_lst)
print(all_names)

# Helper Functions

## General Helper Functions


In [ ]:
def get_one_degree_neighbors(temp2, names, group_lst):
    result = []
    for i in range(len(group_lst)):
        d = copy.copy(temp2)
        for k in range(len(group_lst)):
            if k != i:
                d = d[d[names[k]] == group_lst[k]]
            else:
                d = d[d[names[k]] != group_lst[k]]
        # print(d)
        result.append(d)
    return result

In [ ]:
# compute the pos/neg ration of this neighbor
def compute_neighbors(group_lst, result):
    # compute the ratio of positive and negative records
    start2 = time.time()
    pos = 0
    neg = 0
    for r in result:
        total  = r['cnt'].sum()
        r = r[r[compas_y] == 1]
        pos += r['cnt'].sum()
        neg += total - r['cnt'].sum()
    if(neg == 0):
        return (pos, neg, -1)
    end2 = time.time()
    # print("The time to compute the neighbor counts for " +  str(group_lst) +" is " + str(end2-start2))
    return(pos, neg, pos/neg)

In [ ]:
def compute_diff_add_and_remove(group_lst, temp2, need_positive_or_negative, label, names):
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
        d = d[d[names[i]] == group_lst[i]]
        # print(len(d))
    total =  d['cnt'].sum()
    # Total here was 0: here, errors when this is commented out
    if total == 0:
      return -1
    d = d[d[label] == 1]
    pos = d['cnt'].sum()
    # print(d, group_lst)
    neg = total - pos
    result = get_one_degree_neighbors(temp2,names, group_lst)
    neighbors = compute_neighbors(group_lst, result)
    if(need_positive_or_negative == 1):
        # need pos
        x = Symbol('x')
        # print(pos, neg , neighbors[2])
        try:
          diff = solve((pos + x)/ (neg - x) - neighbors[2])[0]
        except:
          return -1
        # print(solve((pos + x)/ (neg - x) - neighbors[2]))
    else:
        #need negative
        x = Symbol('x')
        try:
          diff = solve((pos - x)/ (neg + x) - neighbors[2])[0]
        except:
          return -1
    print(neighbors[2],pos, neg, diff)
    return diff

In [ ]:
def compute_diff_add(group_lst, temp2, names, label_y, need_positive_or_negative):
    # print("compute diff")
    d = copy.copy(temp2)
    # print("here", d, names)
    for i in range(len(group_lst)):
        # print(d, group_lst[i])
        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label_y] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
    result = get_one_degree_neighbors(temp2, names, group_lst)
    neighbors = compute_neighbors(group_lst, result)
    if(need_positive_or_negative == 1):
        # need pos
        # if neg == 0:
        #     # to avoid zero neg
        #     neg = 1
        x = Symbol('x')
        diff = solve((pos + x)/ neg -  neighbors[2])[0]
        # enough = neighbors[2] * neg
        # diff = round_int(enough - pos)
        print(neighbors[2], pos, neg, diff)
    else:
        #need negative
        # if(pos == 0):
        #     diff  = 0
        #     enough = 0
        # else:
        # enough = pos / (neighbors[2] )
        x = Symbol('x')
        diff = solve(pos/ (neg + x) -  neighbors[2])[0]
    print(neighbors[2], pos, neg, diff)
    return diff

def compute_diff_remove(group_lst, temp2, names, label_y, need_positive_or_negative):
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
        # print(d, group_lst[i])
        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label_y] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
    result = get_one_degree_neighbors(temp2, names, group_lst)
    neighbors = compute_neighbors(group_lst, result)
    if(need_positive_or_negative == 1):
        # need pos, remove some neg
        x = Symbol('x')
        try:
          diff = solve( pos/ (neg - x) -  neighbors[2])[0]
        except:
          return -1
        # enough = neighbors[2] * neg
        # diff = round_int(enough - pos)
        print(neighbors[2], pos, neg, diff)
    else:
        #need negative
        x = Symbol('x')
        # diff = solve((pos -x )/ neg -  neighbors[2])[0]
        try:
          diff = solve((pos -x )/ neg -  neighbors[2])[0]
        except:
          return -1
        print(neighbors[2], pos, neg, diff)
    return diff


In [ ]:
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer

def div_results(db, remedy, algo):
  columns_compas.extend([compas_y, "predicted"])

  df = pd.DataFrame(test_set, columns = columns_compas)
  # print(test_set.columns)
  print(columns_compas)
  print(df.columns)
  print(columns_compas)

  columns_compas.remove(compas_y)
  columns_compas.remove('predicted')
  class_map={'N': 0, 'P': 1}

  min_sup=0.1

  # min_sup = 0.05

  fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
  FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
  from divexplorer.FP_Divergence import FP_Divergence
  fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
  fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
  fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
  INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
  INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
  INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]

  K=10
  pd.options.display.max_rows = 200
  d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
  # summerization
  eps=0.01

  # d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ].head(K)
  # d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2].head(K)
  # d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3].head(K)

  d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
  d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
  d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3]

  d= d[d['d_fpr'] > 0]
  d2= d2[d2['d_fnr'] > 0]

  dfpr = fairness_score_computation(d, 'd_fpr')
  dfnr = fairness_score_computation(d2, 'd_fnr')
  dacc = fairness_score_computation(d3, 'd_accuracy')
  print(dfpr)
  print(dfnr)
  print(dacc)
  accuracy = accuracy_score(test_label, test_predict)
  print("accuracy is " , accuracy)
  writelist = [db,remedy,algo, dfpr, dfnr, dacc, accuracy]
  with open('GC_results.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(writelist)
  print(writelist)
  return d,d2,d3

## Optimized Helper Function


In [ ]:
# helper function for optimized
def compute_neighbors_opt(group_lst,lst_of_counts, pos, neg):
    #start2 = time.time()
    times = len(group_lst)
    pos_cnt = 0
    neg_cnt = 0
    for i in range(times):
        df_groupby = lst_of_counts[i]
        temp_group_lst_pos = copy.copy(group_lst)
        temp_group_lst_neg = copy.copy(group_lst)
        del temp_group_lst_pos[i]
        del temp_group_lst_neg[i]
        # count positive
        temp_group_lst_pos.append(1)
        group_tuple_pos = tuple(temp_group_lst_pos)
        if group_tuple_pos in df_groupby.keys():
            pos_cnt += df_groupby[group_tuple_pos]
        else:
            pos_cnt += 0
        # count negative
        temp_group_lst_neg.append(0)
        group_tuple_neg = tuple(temp_group_lst_neg)
        if group_tuple_neg in df_groupby.keys():
            neg_cnt += df_groupby[group_tuple_neg]
        else:
            neg_cnt += 0
    pos_val = pos_cnt - times* pos
    neg_val = neg_cnt - times* neg
    #end2 = time.time()
    #print("The time to compute the neighbor counts for " +  str(group_lst) +" is " + str(end2-start2))
    if neg_val == -1 or (neg_val == 0 and pos_val == 0):
        return (pos_val, neg_val, -1)
    if pos_val == 0 or neg_val == 0:
        return (pos_val, neg_val, 0)
    return (pos_val, neg_val, pos_val/neg_val)

In [ ]:
# get the list of neighbors
def get_one_degree_neighbors_opt(group_lst):
    result = []
    for i in range(len(group_lst)):
        d = copy.copy(group_lst)
        d[i] = 'x'
        result.append(d)
    return result

In [ ]:
def determine_problematic_opt(group_lst, names, temp2, lst_of_counts, label, threshold= 0.3):
    #0: ok group, 1: need negative records, 2: need positive records
    d = copy.copy(temp2)
    for i in range(len(group_lst)):
        d = d[d[names[i]] == group_lst[i]]
    total =  d['cnt'].sum()
    d = d[d[label] == 1]
    pos = d['cnt'].sum()
    neg = total - pos
    neighbors = compute_neighbors_opt(group_lst,lst_of_counts, pos, neg)
    if(neighbors[2] == -1):
        # there is no neighbors
        return 0
    if(total > filter_count):
        # need to be large enough, need to adjust with different datasets.
        if neg == 0:
            if (pos > neighbors[2]):
                return 1
            if(pos <= neighbors[2]):
                return 0
        if (pos/(neg) - neighbors[2] > threshold):
            # too many positive records
            return 1
        if (neighbors[2] - pos/(neg) > threshold):
            return 2
    return 0

In [ ]:
def compute_problematic_opt(temp2, temp_g, names, label, lst_of_counts):
    need_pos = []
    need_neg = []
    for index, row in temp_g.iterrows():
        group_lst = []
        for n in names:
            group_lst.append(row[n])
        problematic = determine_problematic_opt(group_lst, names, temp2, lst_of_counts,label)
#         #print(problematic)
        if(problematic == 1):
            if group_lst not in need_neg:
                need_neg.append(group_lst)
        if(problematic == 2):
            if group_lst not in need_pos:
                need_pos.append(group_lst)
    return need_pos, need_neg

In [ ]:
# build the list of X00
def compute_lst_of_counts(temp, names, label):
    # get the list of group-by attributes
    lst_of_counts = []
    for i in range(len(names)):
        grp_names = copy.copy(names)
        del grp_names[i]
        grp_names.append(label)
        temp_df = temp.groupby(grp_names)['cnt'].count()
        lst_of_counts.append(temp_df)
    return lst_of_counts

def get_tuple(group_lst):
    return tuple(group_lst)


def get_temp_g(train_set, names, y_label):
  names2 = copy.deepcopy(names)
  names2.append(y_label)
  temp = train_set[names2]
  temp['cnt'] = 0
  temp_g = temp.groupby(names)['cnt'].count().reset_index()
  return temp, temp_g

# Preferential Sampling

In [ ]:
from sklearn.naive_bayes import MultinomialNB
def pref_sampling_opt(train_set, cols_given, label, need_pos, need_neg):
    if len(need_pos)+ len(need_neg) > 0:
        temp_train_x = pd.DataFrame(train_set, columns = columns_all)
        temp_train_label = pd.DataFrame(train_set, columns = [label])
        temp_train_label = temp_train_label[label]
        temp_train_label = temp_train_label.astype('int')
        mnb = MultinomialNB()
        mnb = mnb.fit(temp_train_x, temp_train_label)
        probs = mnb.predict_proba(temp_train_x)[:,0]
        train_set["prob"] = abs(probs - 0.5)
        # get the set of
    new_train_set = pd.DataFrame(columns = list(train_set.columns))
    updated_pos = 0
    for i in need_pos:
        # needs to updated more positive records
        # print(i)
        temp_df = copy.deepcopy(train_set)
        for n in range(len(i)):
          temp_df = temp_df[temp_df[cols_given[n]] == i[n]]
        # update the skew and diff
        idx = list(temp_df.index)
        train_set.loc[idx, 'skewed'] = 1
        idx_pos = list(temp_df[(getattr(temp_df, label) == 1)].index)
        if(len(idx_pos) == 0):
          # if there is no positive
          idx_neg = list(temp_df[(getattr(temp_df, label) == 0)].index)
          neg_ranked = train_set.loc[idx_neg].sort_values(by="prob", ascending=True)
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          continue
        idx_neg = list(temp_df[(getattr(temp_df, label) == 0)].index)
        pos_ranked = train_set.loc[idx_pos].sort_values(by="prob", ascending=True)
        neg_ranked = train_set.loc[idx_neg].sort_values(by="prob", ascending=True)
        diff = compute_diff_add_and_remove(i, temp2,  1, compas_y, names)
        if diff == -1:
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          continue
        train_set.loc[idx, 'diff'] = int(diff)
        cnt = int(train_set.loc[idx_pos[0]]["diff"])
        updated_pos += cnt * 2
        # add more records when there are not enough available records
        new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
        temp_cnt = cnt
        if len(pos_ranked) >= temp_cnt:
            new_train_set = pd.concat([new_train_set,pos_ranked[0:cnt]], ignore_index=True)
        else:
            while(temp_cnt > 0 ):
                new_train_set = pd.concat([new_train_set,pos_ranked[0:temp_cnt]], ignore_index=True)
            # duplicate the dataframe
                temp_cnt = temp_cnt - len(pos_ranked)
        # duplicate the top cnt records from the pos
        # remove the top cnt records from the neg
        if cnt == 0:
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          # print("+++++++++++++++++++++++++++++++++++++++++++++")
          # print(i)
          # print(len(pos_ranked)+cnt)
          # print(len(neg_ranked))
        else:
          new_train_set = pd.concat([new_train_set, neg_ranked[cnt-1:-1]], ignore_index=True)
          # print("+++++++++++++++++++++++++++++++++++++++++++++")
          # print(i)
          # print(len(pos_ranked)+cnt)
          # print(len(neg_ranked[cnt-1:-1]))
    print("updated {} positive records".format(str(updated_pos)))
    updated_neg = 0
    # adding more records to the need_neg set
    for i in need_neg:
        # print(i)
        # list of idx belongs to this group
        temp_df = copy.deepcopy(train_set)
        for n in range(len(i)):
          temp_df = temp_df[temp_df[cols_given[n]] == i[n]]
        # update the skew and diff
        idx = list(temp_df.index)
        train_set.loc[idx, 'skewed'] = 1
        idx_pos = list(temp_df[(getattr(temp_df, label) == 1)].index)
        idx_neg = list(temp_df[(getattr(temp_df, label) == 0)].index)
        if(len(idx_neg) == 0):
          pos_ranked = train_set.loc[idx_pos].sort_values(by="prob", ascending=True)
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          continue
        pos_ranked = train_set.loc[idx_pos].sort_values(by="prob", ascending=True)
        neg_ranked = train_set.loc[idx_neg].sort_values(by="prob", ascending=True)
        diff = compute_diff_add_and_remove(i, temp2, 0, compas_y, names)
        if diff == -1:
          new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          continue
        train_set.loc[idx, 'diff'] = int(diff)
        cnt = int(train_set.loc[idx_pos[0]]["diff"])
        updated_neg += cnt * 2
        # add more records when there are not enough available records
        new_train_set = pd.concat([new_train_set, neg_ranked], ignore_index=True)
        temp_cnt = cnt
        if len(neg_ranked) >= temp_cnt:
            new_train_set = pd.concat([new_train_set,neg_ranked[0:cnt]], ignore_index=True)
        else:
            while(temp_cnt > 0 ):
                new_train_set = pd.concat([new_train_set,neg_ranked[0:temp_cnt]], ignore_index=True)
            # duplicate the dataframe
                temp_cnt = temp_cnt - len(neg_ranked)
        # duplicate the top cnt records from the pos
        # remove the top cnt records from the neg
        if cnt ==0:
          new_train_set = pd.concat([new_train_set, pos_ranked], ignore_index=True)
          # print("+++++++++++++++++++++++++++++++++++++++++++++")
          # print(i)
          # print(len(neg_ranked)+cnt)
          # print(len(pos_ranked))
        else:
          new_train_set = pd.concat([new_train_set, pos_ranked[cnt-1:-1]], ignore_index=True)
          # print("+++++++++++++++++++++++++++++++++++++++++++++")
          # print(i)
          # print(len(neg_ranked)+cnt)
          # print(len(pos_ranked[cnt-1:-1]))
        #print(len(new_train_set[new_train_set['income'] == 1]), len(new_train_set[new_train_set['income'] == 0]))
        # print(train_set.loc[idx_neg])
    print("updated {} negative records".format(str(updated_neg)))
    # add the other irrelavant items:
    idx_irr = list(train_set[train_set['skewed'] == 0].index)
    irr_df = train_set.loc[idx_irr]
    new_train_set = pd.concat([new_train_set, irr_df], ignore_index=True)
    print("The new dataset contains {} rows.".format(str(len(new_train_set))))
    new_train_set.reset_index()
    return new_train_set


In [ ]:
# unfair_group, unfair_names, skew_candidates, unfair_dict = get_unfair_group([])
# print(unfair_group, unfair_names, skew_candidates, unfair_dict)
# combos_dict, all_names = candidate_groups(skew_candidates, unfair_dict, columns_compas, unfair_names)
# print(combos_dict, all_names)

# # the
# print(list(all_names.keys())[len(columns_compas)+1:])
# all_names_lst = list(all_names.keys())[len(columns_compas)+1:]
# all_names_lst.reverse()
# print(all_names_lst)

def find_top(all_names):
  all_names_lst_top = []
  for all in range(len(all_names)):
    if len(all_names[all]) == 1: # CHANGED HERE
      all_names_lst_top.append(all)
  return all_names_lst_top

## Run Algorithm Lattice


In [ ]:

#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in all_names_lst:
# for a in [all_names_lst[0]]: # leaf
# for a in [all_names_lst[-1:x]]: # top
  print("/////////////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  # print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  print("started pref sampling")
  print(names)
  new_train_data = pref_sampling_opt(new_train_data, names, compas_y, need_pos, need_neg)
  # print(new_train_data)
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
# print(pref_sampling_mod(train_x, train_set, train_label, names, compas_y, need_pos, need_neg))

### Results Lattice

In [ ]:
print("dt")
griddt.fit(new_train_x, new_train_label)
print("best", griddt.best_score_)
test_predict = griddt.predict(test_x)
test_set['predicted'] = test_predict

In [ ]:
# run divexplorer to find unfair groups
# run divexplorer to find unfair groups
class_map={'N': 0, 'P': 1}
from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer


######
# Yin: by consideri the unfair subgroups, we only consider the subgroups given by the protected attributes.

columns_compas.extend([compas_y, "predicted"])

df = pd.DataFrame(test_set, columns = columns_compas)
# print(test_set.columns)
print(columns_compas)
print(df.columns)
print(columns_compas)

columns_compas.remove(compas_y)
columns_compas.remove('predicted')

######

min_sup=0.1
# min_sup = 0.05
# min_sup = 0.01

fp_diver=FP_DivergenceExplorer(df,compas_y, "predicted", class_map=class_map)
FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
from divexplorer.FP_Divergence import FP_Divergence
fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
fp_divergence_fnr=FP_Divergence(FP_fm, "d_fnr")
fp_divergence_acc=FP_Divergence(FP_fm, "d_accuracy")
  # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]
INFO_VIZ2=["support", "itemsets",  fp_divergence_fnr.metric, fp_divergence_fnr.t_value_col]
INFO_VIZ3=["support", "itemsets",  fp_divergence_acc.metric, fp_divergence_acc.t_value_col]
eps=0.01
K=1000
d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
d3 = fp_divergence_acc.getDivergence(th_redundancy=eps)[INFO_VIZ3]

# dfpr = fairness_score_computation(d, 'd_fpr')
# dfnr = fairness_score_computation(d2, 'd_fnr')
# dacc = fairness_score_computation(d3, 'd_accuracy')

pd.options.display.max_rows = 200
d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ]
# summerization

d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ]
d= d[d['d_fpr'] > 0]
# d, list(d["itemsets"].iloc[0])

# d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2].head(K)
# d2= d2[d2['d_fnr'] > 0]
# d2, list(d2["itemsets"].iloc[0])

In [ ]:
d

In [ ]:
dfpr = fairness_score_computation(d, 'd_fpr')
dfpr

In [ ]:
d2 = fp_divergence_fnr.getDivergence(th_redundancy=eps)[INFO_VIZ2]
d2= d2[d2['d_fnr'] > 0]
d2, list(d2["itemsets"].iloc[0])

In [ ]:
print(fairness_score_computation(d, 'd_fpr'))
print(fairness_score_computation(d2, 'd_fnr'))
print(fairness_score_computation(d3, 'd_accuracy'))
d,d1,d2 = div_results("German Credit", "Preferential Sampling-Lattice","DT")

In [ ]:
# d,d1,d2 = div_results("German Credit", "Preferential Sampling-Lattice","Decision Tree")
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("German Credit", "Preferential Sampling-Lattice","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("German Credit", "Preferential Sampling-Lattice","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("German Credit", "Preferential Sampling-Lattice","SVM")

## Run Algorithm Leaf

In [ ]:

#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in [all_names_lst[0]]:
# for a in [all_names_lst[0]]: # leaf
# for a in [all_names_lst[-1:x]]: # top
  print("/////////////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  # print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  print("started pref sampling")
  new_train_data = pref_sampling_opt(new_train_data, names, compas_y, need_pos, need_neg)
  # print(new_train_data)
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
# print(pref_sampling_mod(train_x, train_set, train_label, names, compas_y, need_pos, need_neg))

### Results Leaf

In [ ]:
test_set['predicted'] = test_predict
d, d2, d3 = div_results("German Credit", "Preferential Sampling-Leaf","DT")

In [ ]:
# d,d1,d2 = div_results("German Credit", "Preferential Sampling-Leaf","RF")
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("German Credit", "Preferential Sampling-Leaf","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("German Credit", "Preferential Sampling-Leaf","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("German Credit", "Preferential Sampling-Leaf","SVM")

## Run Algorithm Top

In [ ]:

#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)
all_names_lst_top = find_top(all_names)
#iterate over all the names to get the temp2 df for each name
for a in all_names_lst_top:
# for a in [all_names_lst[0]]: # leaf
# for a in [all_names_lst[-1:x]]: # top
  print("/////////////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  # print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  print("started pref sampling")
  new_train_data = pref_sampling_opt(new_train_data, names, compas_y, need_pos, need_neg)
  # print(new_train_data)
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
# print(pref_sampling_mod(train_x, train_set, train_label, names, compas_y, need_pos, need_neg))

### Results Top

In [ ]:
test_set['predicted'] = test_predict
d, d2, d3 = div_results("German Credit", "Preferential Sampling-Top","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("German Credit", "Preferential Sampling-Top","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("German Credit", "Preferential Sampling-Top","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("German Credit", "Preferential Sampling-Top","SVM")

# Duplication

In [ ]:
def round_int(x):
    if x in [float("-inf"),float("inf")]: return 0
    return int(round(x))


def make_duplicate(d, group_lst, diff, label_y, names, need_positive_or_negative):
    #print("make samples")
    selected = copy.deepcopy(d)
    print("names ", names, group_lst)
    for i in range(len(group_lst)):
        att_name = names[i]
        selected = selected[(selected[att_name] == group_lst[i])]
    selected = selected[(selected[label_y] == need_positive_or_negative)]
    # print("=============")
    # print(group_lst, names)
    # print(selected)
    if len(selected) == 0:
        return pd.DataFrame()
    # print(len(temp))
    # randomly generated diff samples:
    # print(len(selected), diff)
    while(len(selected) < diff):
        # duplicate the dataframe
        select_copy = selected.copy(deep=True)
        selected = pd.concat([selected, select_copy])
        # print(len(selected))
        #generated = temp
        # the number needed is more than the not needed numbers.
    #print(len(temp))
    generated = selected.sample(n = diff, replace = False, axis = 0)
    # print(generated)
    return generated


def naive_duplicate(d, temp2, names, need_pos, need_neg, label_y):
    # add more records for all groups
    # The smote algorithm to boost the coverage
    for r in need_pos:
        # print("adding more positive")
    # add more positive records
        # determine how many points to add
        print("pos_vals", r)
        diff = compute_diff_add(r, temp2, names, label_y, 1)
        diff = round_int(diff)
        # add more records
        print("Adding " + str(diff) +" positive records")
        samples_to_add = make_duplicate(d, r, diff, label_y, names, need_positive_or_negative = 1)
        d = pd.concat([d, samples_to_add], ignore_index=True)
    for k in need_neg:
        print("neg_vals", k)
        # print("adding more negative")
        diff = compute_diff_add(k, temp2, names, label_y, need_positive_or_negative = 0)
        diff = round_int(diff)
        print("Adding " + str(diff) +" negative records")
        samples_to_add = make_duplicate(d, k, diff, label_y, names, need_positive_or_negative = 0)
        d = pd.concat([d, samples_to_add], ignore_index=True)
    return d

## Run Algorithm Lattice

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in all_names_lst:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  # start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  # end = time.time()
  # excute_time = end - start
  # print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  print("started duplication")
  new_train_data = naive_duplicate(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  # new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
  # new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
  # new_train_label = new_train_label[compas_y]
  # new_train_label = new_train_label.astype('int')
  # grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
  # grid_new.fit(new_train_x, new_train_label)
  # test_predict = grid_new.predict(test_x)
  # print("new trainset length", len(new_train_x))
  # print("fpr and fnr")
  # print(fpr_onegroup(list(test_label), test_predict))
  # print(fnr_onegroup(list(test_label), test_predict))
  print(compas_y, new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))

In [ ]:
test_set['predicted'] = test_predict

### Results Lattice


In [ ]:
# # run divexplorer to find unfair groups
# class_map={'N': 0, 'P': 1}
# from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer

# #min_sup=0.1

# min_sup = 0.05

# fp_diver=FP_DivergenceExplorer(test_set,"class", "predicted", class_map=class_map)
# FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
# from divexplorer.FP_Divergence import FP_Divergence
# fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
# # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
# INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]

# K=10
# pd.options.display.max_rows = 200
# d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# # summerization
# eps=0.01

# d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ].head(K)

# d
d, d2, d3 = div_results("Bar", "Duplication-Lattice","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Duplication-Lattice","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Duplication-Lattice","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Duplication-Lattice","SVM")

In [ ]:
def fairness_score_computation(d, metrics):
    sum_of_score = 0
    for idx, row in d.iterrows():
      sum_of_score += row['support'] * row[metrics]
    return sum_of_score

print(fairness_score_computation(d, 'd_fpr'))


## Run Algorithm Leaf

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in [all_names_lst[0]]:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  # start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  # end = time.time()
  # excute_time = end - start
  # print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  print("started duplication")
  new_train_data = naive_duplicate(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  # new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
  # new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
  # new_train_label = new_train_label[compas_y]
  # new_train_label = new_train_label.astype('int')
  # grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
  # grid_new.fit(new_train_x, new_train_label)
  # test_predict = grid_new.predict(test_x)
  # print("new trainset length", len(new_train_x))
  # print("fpr and fnr")
  # print(fpr_onegroup(list(test_label), test_predict))
  # print(fnr_onegroup(list(test_label), test_predict))
  print(compas_y, new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))

### Results Leaf

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Duplication-Leaf","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Duplication-Leaf","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Duplication-Leaf","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Duplication-Leaf","SVM")

## Run Algorithm Top

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
all_names_lst_top = find_top(all_names)
for a in all_names_lst_top:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  # start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  # end = time.time()
  # excute_time = end - start
  # print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  print("started duplication")
  new_train_data = naive_duplicate(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  # new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
  # new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
  # new_train_label = new_train_label[compas_y]
  # new_train_label = new_train_label.astype('int')
  # grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
  # grid_new.fit(new_train_x, new_train_label)
  # test_predict = grid_new.predict(test_x)
  # print("new trainset length", len(new_train_x))
  # print("fpr and fnr")
  # print(fpr_onegroup(list(test_label), test_predict))
  # print(fnr_onegroup(list(test_label), test_predict))
  print(compas_y, new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))

### Results Top

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Duplication-Top","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Duplication-Top","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Duplication-Top","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Duplication-Top","SVM")

#Down-sampling

In [ ]:
def round_int(x):
    if x in [float("-inf"),float("inf")]: return 0
    return int(round(x))


def make_remove(d, group_lst, diff, names, label_y, need_positive_or_negative):
    #print("make samples")
    temp = copy.deepcopy(d)
    for i in range(len(group_lst)):
        att_name = names[i]
        temp = temp[(temp[att_name] == group_lst[i])]
    temp = temp[(temp[label_y] == need_positive_or_negative)]
    # randomly generated diff samples
        #generated = temp
        # the number needed is more than the not needed numbers.
    # print(len(temp))
    # print(diff)
    if(diff>len(temp)):
        diff = len(temp)
    generated = temp.sample(n = diff, replace = False, axis = 0)
    #print(generated.index)
    return generated.index


def naive_downsampling(d, temp2, names, need_pos, need_neg, label_y):
    # add more records for all groups
    # The smote algorithm to boost the coverage
    for r in need_pos:
        print("removing more negative")
    # add more positive records
        # determine how many points to add
        print(r)
        diff = compute_diff_remove(r, temp2, names, label_y, need_positive_or_negative = 1)
        diff = round_int(diff)
        if diff == -1:
          continue
        # add more records
        print("Removed " + str(diff) +" negative records")
        samples_to_remove = make_remove(d, r, diff, names, label_y, need_positive_or_negative = 0)
        d.drop(index  = samples_to_remove, inplace = True)
        # print(len(d))
    for k in need_neg:
        print(k)
        diff = compute_diff_remove(k, temp2, names, label_y, need_positive_or_negative = 0)
        diff = round_int(diff)
        if diff == -1:
          continue
        print("Removed " + str(diff) +" positive records")
        samples_to_remove = make_remove(d, k, diff, names, label_y, need_positive_or_negative = 1)
        d.drop(index  = samples_to_remove, inplace = True)
    return d

## Run Algorithm Lattice

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in all_names_lst:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  # print(temp_g)
  # print(temp2, names)
  # print(temp)
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  # print("started duplication")
  new_train_data = naive_downsampling(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
# new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
# new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
# new_train_label = new_train_label[compas_y]
# new_train_label = new_train_label.astype('int')
# grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
# grid_new.fit(new_train_x, new_train_label)
# test_predict = grid_new.predict(test_x)
# print("new trainset length", len(new_train_x))
# print("fpr", fpr_onegroup(list(test_label), test_predict))
# print("fnr", fnr_onegroup(list(test_label), test_predict))

### Results Lattice

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Downsampling-Lattice","DT")

# # run divexplorer to find unfair groups
# class_map={'N': 0, 'P': 1}
# from divexplorer.FP_DivergenceExplorer import FP_DivergenceExplorer

# #min_sup=0.1

# min_sup = 0.05

# fp_diver=FP_DivergenceExplorer(test_set,"class", "predicted", class_map=class_map)
# FP_fm=fp_diver.getFrequentPatternDivergence(min_support=min_sup, metrics=["d_fpr", "d_fnr", "d_accuracy"])
# from divexplorer.FP_Divergence import FP_Divergence
# fp_divergence_fpr=FP_Divergence(FP_fm, "d_fpr")
# # fp_divergence_fpr=FP_Divergence(FP_fm, "d_fnr")
# INFO_VIZ=["support", "itemsets",  fp_divergence_fpr.metric, fp_divergence_fpr.t_value_col]

# K=10
# pd.options.display.max_rows = 200
# d = fp_divergence_fpr.getDivergence(th_redundancy=0)[INFO_VIZ].head(K)
# # summerization
# eps=0.01

# d = fp_divergence_fpr.getDivergence(th_redundancy=eps)[INFO_VIZ].head(K)

# d


In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Downsampling-Lattice","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Downsampling-Lattice","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Downsampling-Lattice","SVM")

## Run Algorithm Leaf

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in [all_names_lst[0]]:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  # print(temp_g)
  # print(temp2, names)
  # print(temp)
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  # print("started duplication")
  new_train_data = naive_downsampling(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
# new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
# new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
# new_train_label = new_train_label[compas_y]
# new_train_label = new_train_label.astype('int')
# grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
# grid_new.fit(new_train_x, new_train_label)
# test_predict = grid_new.predict(test_x)
# print("new trainset length", len(new_train_x))
# print("fpr", fpr_onegroup(list(test_label), test_predict))
# print("fnr", fnr_onegroup(list(test_label), test_predict))

### Results Leaf

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Downsampling-Leaf","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Downsampling-Leaf","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Downsampling-Leaf","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Downsampling-Leaf","SVM")

## Run Algorithm Top

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
all_names_lst_top = find_top(all_names)
for a in all_names_lst_top:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  # print(temp_g)
  # print(temp2, names)
  # print(temp)
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  # print("started duplication")
  new_train_data = naive_downsampling(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
# new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
# new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
# new_train_label = new_train_label[compas_y]
# new_train_label = new_train_label.astype('int')
# grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
# grid_new.fit(new_train_x, new_train_label)
# test_predict = grid_new.predict(test_x)
# print("new trainset length", len(new_train_x))
# print("fpr", fpr_onegroup(list(test_label), test_predict))
# print("fnr", fnr_onegroup(list(test_label), test_predict))

### Results Top

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Downsampling-Top","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Downsampling-Top","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Downsampling-Top","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Downsampling-Top","SVM")

# Massaging

In [ ]:
from sklearn.naive_bayes import MultinomialNB
def round_int(x):
    if x in [float("-inf"),float("inf")]: return 0
    return int(round(x))

def get_depromotion(d, diff, group_lst, names, label_y, flag_depro):
    # double check
    print("start depromotion")
    # print(len(d))
    print(names)
    # input_test = d.drop(columns = [label_y])
    # COMPAS COLUMNS = ]]
    # input_test = pd.DataFrame(train_set, columns = names)
    input_test = pd.DataFrame(d, columns = columns_compas)
    # should it be d instead?
    # print(group_lst, names)
    clf = MultinomialNB()
    # temp_train_label = pd.DataFrame(train_set, columns = [label_y])
    temp_train_label = pd.DataFrame(d, columns = [label_y])
    temp_train_label = temp_train_label[label_y]
    temp_train_label = temp_train_label.astype('int')
    clf = clf.fit(input_test, temp_train_label)
    prob  = clf.predict_proba(input_test)[:,0]
    select = copy.deepcopy(d)
    select['prob'] = prob # the higher the probablity is, the more likely for it to be 0
    # filter out those belongs to this group
    for i in range(len(group_lst)):
        att_name = names[i]
        select = select[(select[att_name] == group_lst[i])]
    select = select[(select[label_y] == flag_depro)]
    # rank them according to the probability
    # filp the records and remove the records from d
    if (flag_depro == 0):
        select.sort_values(by="prob", ascending=True, inplace=True)
        select[label_y] = 1
    else:
        select.sort_values(by="prob", ascending=False, inplace=True)
        select[label_y] = 0
    head = select.head(diff)
    index_list = []
    index_list = list(head.index)
    d.drop(index_list,inplace = True)
    head.drop(columns = ['prob'],inplace = True)
    #print(head.head())
    #print(d.head())
    return head



def naive_massaging(d, temp2, names, need_pos, need_neg,label_y):
    # add more records for all groups
    # The smote algorithm to boost the coverage
    for r in need_pos:
        print("adding more positive")
    # add more positive records
        # determine how many points to add
        print(r)
        diff = compute_diff_add_and_remove(r, temp2, 1, label_y, names)
        diff =  round_int(diff)
        # add more records
        #0 for promotion
        samples_to_add = get_depromotion(d, diff, r, names, label_y, flag_depro = 0)
        print("Changed " + str(len(samples_to_add)) +" records")
        d = pd.concat([d, samples_to_add])
        print(len(d))
    for k in need_neg:
        print(k)
        print("adding more negative")
        diff = compute_diff_add_and_remove(k, temp2, 0, label_y, names)
        diff =  round_int(diff)
        #1 for demotion
        samples_to_add = get_depromotion(d, diff, k, names, label_y, flag_depro = 1)
        print("Changed " + str(len(samples_to_add)) +" records")
        d = pd.concat([d, samples_to_add])
        print(len(d))
    return d

## Run Algorithm Lattice

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in all_names_lst:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  # print(temp_g)
  # print(temp2, names)
  # print(temp)
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  # print("started duplication")
  new_train_data = naive_massaging(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  # print(new_train_data)
  # new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
  # new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
  # new_train_label = new_train_label[compas_y]
  # new_train_label = new_train_label.astype('int')
  # grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
  # grid_new.fit(new_train_x, new_train_label)
  # test_predict = grid_new.predict(test_x)
  # print("new trainset length", len(new_train_x))
  # print("fpr", fpr_onegroup(list(test_label), test_predict))
  # print("fnr", fnr_onegroup(list(test_label), test_predict))
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))

### Results Lattice

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Massaging-Lattice","DT")


In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Massaging-Lattice","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Massaging-Lattice","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Massaging-Lattice","SVM")

## Run Algorithm Leaf

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
for a in [all_names_lst[0]]:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  # print(temp_g)
  # print(temp2, names)
  # print(temp)
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  # print("started duplication")
  new_train_data = naive_massaging(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  # print(new_train_data)
  # new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
  # new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
  # new_train_label = new_train_label[compas_y]
  # new_train_label = new_train_label.astype('int')
  # grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
  # grid_new.fit(new_train_x, new_train_label)
  # test_predict = grid_new.predict(test_x)
  # print("new trainset length", len(new_train_x))
  # print("fpr", fpr_onegroup(list(test_label), test_predict))
  # print("fnr", fnr_onegroup(list(test_label), test_predict))
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))

### Results Leaf

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Massaging-Leaf","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Massaging-Leaf","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Massaging-Leaf","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Massaging-Leaf","SVM")

## Run Algorithm Top

In [ ]:
#get all of the candidate groups possible with the combos and names

new_train_data = copy.deepcopy(train_set)

#iterate over all the names to get the temp2 df for each name
all_names_lst_top = find_top(all_names)
for a in all_names_lst_top:
  print("?????/////")
  print(a)
  temp2, names = get_temp(new_train_data, all_names[a], compas_y)
  temp, temp_g = get_temp_g(new_train_data, names, compas_y)
  temp_g = temp_g[temp_g['cnt'] > filter_count]
  # print(temp_g)
  # print(temp2, names)
  # print(temp)
  lst_of_counts = compute_lst_of_counts(temp, names, compas_y)
  start = time.time()
  need_pos, need_neg = compute_problematic_opt(temp2, temp_g, names, compas_y, lst_of_counts)
  end = time.time()
  excute_time = end - start
  print("The time to compute unfair group is {}".format(str(excute_time)))
  print("The sets of need pos and neg are")
  print(need_pos)
  print(need_neg)
  new_train_data['skewed'] = 0
  new_train_data["diff"] = 0
  # print("started duplication")
  new_train_data = naive_massaging(new_train_data, temp2, names, need_pos, need_neg, compas_y)
  # print(new_train_data)
  # new_train_x = pd.DataFrame(new_train_data, columns = columns_compas)
  # new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
  # new_train_label = new_train_label[compas_y]
  # new_train_label = new_train_label.astype('int')
  # grid_new = DecisionTreeClassifier(criterion="entropy",max_depth=6, random_state=17)
  # grid_new.fit(new_train_x, new_train_label)
  # test_predict = grid_new.predict(test_x)
  # print("new trainset length", len(new_train_x))
  # print("fpr", fpr_onegroup(list(test_label), test_predict))
  # print("fnr", fnr_onegroup(list(test_label), test_predict))
  print(new_train_data[compas_y].value_counts())
new_train_x = pd.DataFrame(new_train_data, columns = columns_all)
new_train_label = pd.DataFrame(new_train_data, columns = [compas_y])
new_train_label = new_train_label[compas_y]
new_train_label = new_train_label.astype('int')
grid_new = GridSearchCV(tree.DecisionTreeClassifier(), param_grid = param, cv = 6)
grid_new.fit(new_train_x, new_train_label)
print("model accuracy")
print("best", grid_new.best_score_)
test_predict = grid_new.predict(test_x)
print("fpr and fnr")
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))

### Results Top

In [ ]:
test_set['predicted'] = test_predict
d,d2,d3 = div_results("Bar", "Massaging-Top","DT")

In [ ]:
# d,d1,d2 = div_results()
print()
print("rf")
gridrf.fit(new_train_x, new_train_label)
print("best", gridrf.best_score_)
test_predict = gridrf.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Massaging-Top","RF")

print()
print("logistic")
gridl.fit(new_train_x, new_train_label)
print("best", gridl.best_score_)
test_predict = gridl.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
r,r2,r3 = div_results("Bar", "Massaging-Top","L")

print()
print("svm")
svc.fit(new_train_x, new_train_label)
# print("best", svc.best_score_)
test_predict = svc.predict(test_x)
print("fpr and fnr")
# print(fpr_onegroup(new_labels.tolist(), predict))
# print(fnr_onegroup(new_labels.tolist(), predict))
print(fpr_onegroup(list(test_label), test_predict))
print(fnr_onegroup(list(test_label), test_predict))
test_set['predicted'] = test_predict
s,s2,s3 = div_results("Bar", "Massaging-Top","SVM")